In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pathlib
import pickle
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, precision_score, auc, make_scorer, recall_score, matthews_corrcoef, f1_score, balanced_accuracy_score, average_precision_score

In [2]:
def afsm12_encode_data(data, input_size):
    """
    Takes in fasta sequence and returns encoded/padded data
    """
    residue_dictionary = {"A": 1, "E": 2, "L": 3, "M": 4, "C": 5, "D": 6, "F": 7, "G": 8,
                          "H": 9, "K":10, "N": 11, "P": 12, "Q": 13, "R": 14, "S": 15,
                          "W": 16, "Y": 17, "T": 18, "V": 19, "I": 20}
    
    fasta = list(str(data))
    # Encode data
    for index, value in enumerate(fasta):
        fasta[index] = residue_dictionary[value]
    # Pad data

    # Invert FASTA and make list 200 times the length to avoid edge cases where FASTA is small
    padding = fasta[::-1]*2000
    
    split = int((input_size-len(fasta))/2)
    last_padding_len = input_size - len(fasta) - split

    stop_pos = int(split+len(fasta))
    padding_1 = padding[-split:]
    padding_2 = padding[:last_padding_len]
    fasta = padding_1 + fasta + padding_2
    
    # Reshape data for input
    fasta = np.array(fasta).reshape(-1, input_size, 1)
    # Normalize data by subtracting training mean and dividing by training std. deviation
    fasta = (fasta - 10.108613363425793)/6.034641898334733
    return fasta, split, stop_pos

def afsm3_encode_data(data, input_size):
    """
    Takes in fasta sequence and returns encoded/padded data
    """
    residue_dictionary = {"A": 1, "E": 2, "L": 3, "M": 4, "C": 5, "D": 6, "F": 7, "G": 8,
                          "H": 9, "K":10, "N": 11, "P": 12, "Q": 13, "R": 14, "S": 15,
                          "W": 16, "Y": 17, "T": 18, "V": 19, "I": 20}
    
    fasta = list(str(data))
    # Encode data
    for index, value in enumerate(fasta):
        fasta[index] = residue_dictionary[value]
    # Pad data

    # Invert FASTA and make list 200 times the length to avoid edge cases where FASTA is small
    padding = fasta[::-1]*2000
    
    split = int((input_size-len(fasta))/2)
    last_padding_len = input_size - len(fasta) - split

    stop_pos = int(split+len(fasta))
    padding_1 = padding[-split:]
    padding_2 = padding[:last_padding_len]
    fasta = padding_1 + fasta + padding_2
    
    # Reshape data for input
    fasta = np.array(fasta).reshape(-1, input_size, 1)
    # Normalize data by subtracting training mean and dividing by training std. deviation
    fasta = (fasta - 10.15)/5.98
    return fasta, split, stop_pos


def afsm12_predict_data(fasta, model, input_size):
    """
    Generate prediction for data point. Will return either predicted pae or plddt.
    """

    data, start_pos, stop_pos = afsm12_encode_data(fasta, input_size)
    prediction = model.predict(data).reshape(input_size, 1)
    prediction = prediction[start_pos:stop_pos]
    prediction = [float(i) for i in prediction]

    return prediction


def afsm3_predict_data(fasta, model, input_size):
    """
    Generate prediction for data point. Will return either probability of 
    crystallization.
    """

    data, start_pos, stop_pos = afsm3_encode_data(fasta, input_size)
    prediction = model.predict(data)[0]
    prediction = list(prediction[:,1])
    prediction = prediction[start_pos:stop_pos]
    prediction = [float(i) for i in prediction]

    return prediction

def encode_sequence(fasta):
    
    residue_dictionary = {"A": 1, "E": 2, "L": 3, "M": 4, "C": 5, "D": 6, "F": 7, "G": 8,
                          "H": 9, "K":10, "N": 11, "P": 12, "Q": 13, "R": 14, "S": 15,
                          "W": 16, "Y": 17, "T": 18, "V": 19, "I": 20}
    
    fasta = list(str(fasta))
    # Encode data
    for index, value in enumerate(fasta):
        fasta[index] = int(residue_dictionary[value])
        
    return fasta

def disorder_list(sequence: str) -> float:

    predictions = []
    # generate encodings for sequence
    afsm1_pred = afsm12_predict_data(sequence, afsm1_model, 4096)
    afsm2_pred = list(np.array(afsm12_predict_data(sequence, afsm2_model, 4096))/100.0)
    afsm3_pred = afsm3_predict_data(sequence, afsm3_model, 2048)
    ordinal_list = encode_sequence(sequence)
    # window size of predictions
    win_size = 11

    start, label, stop = 0, int(win_size), int((win_size * 2) + 1)

    while stop < len(sequence) + 1:
        prediction = pirate_model.predict_proba(
            afsm1_pred[start:stop] + afsm2_pred[start:stop] + afsm3_pred[start:stop] +
        ordinal_list[start:stop])[0]
        predictions.append(prediction)

        start += 1
        label += 1
        stop += 1

    return predictions

In [3]:
local_path = pathlib.Path().absolute()
model_path = str(local_path.parents[0])+"/models/"
afsm1_path = model_path+"afsm1"
afsm2_path = model_path+"afsm2"
afsm3_path = model_path+"afsm3"
pirate_path = model_path+"pirate.pkl"
input_size = 4096
presort_input = 2048
afsm1_model = tf.keras.models.load_model(afsm1_path, custom_objects=None, compile=True, options=None)
print("afsm1 loaded")
afsm2_model = tf.keras.models.load_model(afsm2_path, custom_objects=None, compile=True, options=None)
print("afsm2 loaded")
afsm3_model = tf.keras.models.load_model(afsm3_path, custom_objects=None, compile=True, options=None)
print("afsm3 loaded")
pirate_model = pickle.load(open(pirate_path, 'rb'))
print("pirate loaded")

afsm1 loaded
afsm2 loaded
afsm3 loaded
pirate loaded


In [4]:
ids = []
labels = []
features = []

file = open('disorder_pdb.fasta','r')
count = 0
for line in file:
    line = line.replace('\n', ' ').replace('\r', '').replace('>', '')
    if count == 0:
        ids.append(line.strip())
    if count == 1:
        features.append(line.strip())
    if count == 2:
        line = line.strip()
        line = [e for e in line]
        labels.append(line)
        count = 0
        continue
    count += 1
    continue

print(len(ids), len(features), len(labels))

348 348 348


In [5]:
probs = []
ground_truth = []

for count, sequence in enumerate(features):
    
    if len(sequence) < 2048:
        
        afsm1_pred = afsm12_predict_data(sequence, afsm1_model, 4096)
        afsm2_pred = list(np.array(afsm12_predict_data(sequence, afsm2_model, 4096))/100.0)
        afsm3_pred = afsm3_predict_data(sequence, afsm3_model, 2048)
        ordinal_list = encode_sequence(sequence)

        probabilities = disorder_list(sequence)
        
        probs += probabilities

        ground_truth += labels[count][11:-11]
        print(count, len(probabilities), len(labels[count][11:-11]))

C:\Users\GRICHARDSON\AppData\Local\Temp\ipykernel_10696\1885449952.py:72: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prediction = [float(i) for i in prediction]


0 205 205
1 393 393
2 435 435
3 477 477
4 1708 1708
5 1619 1619
6 1836 1836
7 1644 1644
8 549 549
9 741 741
10 1220 1220
11 527 527
12 831 831
13 812 812
14 379 379
15 319 319
16 577 577
17 742 742
18 584 584
19 670 670
20 551 551
21 372 372
22 856 856
23 1752 1752
24 252 252
25 350 350
26 720 720
27 320 320
28 395 395
29 564 564
30 425 425
31 894 894
33 1082 1082
34 569 569
35 392 392
36 1028 1028
37 1236 1236
38 568 568
39 1827 1827
41 1688 1688
42 1452 1452
43 562 562
45 206 206
46 393 393
47 1704 1704
48 589 589
49 1808 1808
50 477 477
51 876 876
52 498 498
53 672 672
54 941 941
55 1662 1662
56 582 582
57 853 853
58 795 795
59 723 723
60 558 558
61 576 576
62 655 655
63 1637 1637
64 411 411
65 959 959
66 1764 1764
67 1726 1726
68 611 611
69 1464 1464
70 1110 1110
71 263 263
72 272 272
73 1482 1482
74 683 683
75 1446 1446
76 176 176
77 818 818
79 306 306
80 297 297
81 838 838
82 460 460
83 671 671
84 1647 1647
85 327 327
87 617 617
88 257 257
89 907 907
90 1091 1091
91 88 88
92 687 

In [6]:
# remove labels and corresponding predictions if the label isn't 1 or 0
for count, value in reversed(list(enumerate(ground_truth))):
    if value == "-":
        del ground_truth[count]
        del probs[count]

for count, e in enumerate(ground_truth):
    ground_truth[count] = int(ground_truth[count])
    probs[count] = float(probs[count])

In [7]:
best_f1 = 0.0
best_threshold = 0
best_preds = []
for threshold in range(1, 1000, 1):
    new_preds = []
    threshold = float(threshold/1000.0)
    for count, prob in enumerate(probs):
        if prob >= threshold:
            new_preds.append(1)
        if prob < threshold:
            new_preds.append(0)
            
    f1 = f1_score(ground_truth, new_preds)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold
        best_preds = new_preds

In [8]:
accuracy = balanced_accuracy_score(ground_truth, best_preds)
f1 = f1_score(ground_truth, best_preds)
mcc = matthews_corrcoef(ground_truth, best_preds)
auc = roc_auc_score(ground_truth, probs)
aps = average_precision_score(ground_truth, probs)
print(f"Accuracy: {accuracy}")
print(f"Fmax: {f1}")
print(f"MCC: {mcc}")
print(f"AUC: {auc}")
print(f"APS: {aps}")
print(classification_report(ground_truth, best_preds))

Accuracy: 0.87427600003662
Fmax: 0.8371832069634355
MCC: 0.7698086021847229
AUC: 0.9332776663191936
APS: 0.9059162567355865
              precision    recall  f1-score   support

           0       0.91      0.95      0.93     71579
           1       0.88      0.80      0.84     32713

    accuracy                           0.90    104292
   macro avg       0.90      0.87      0.88    104292
weighted avg       0.90      0.90      0.90    104292

